

```
# 코드로 형식 지정됨
```

# 2023 Fall EE331 Project


**OBJECTIVE**

You are provided with a simple PyTorch model that classfies images into one of three categories: rock, paper or scissors. The current model uses just one fully connected (FC) layer for classfication. Your task is to do the following:

1. Model Design


> Replace the single FC layer in the '' class with a neural network architecture of your choice. Consider using combinations of layers such as multiple FC layers, convolutional layers, activation functions and dropout layers.


2. Training


> Once the model is designed,

use the providced training loop to train the model. Adjust the hyperparameters if necessary. You may change the provided dataset with a dataset of your choice. However, this must be stated in the report you are going to write.


You are free to change the code sections 1~????.

For more detail on the report, refer to the project pdf file uploaded in KLMS

0. Libraries

In [1]:
import os
from typing_extensions import dataclass_transform
import pandas as pd
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms
from torchvision.utils import make_grid
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset, random_split
import torch.nn.functional as F
import glob
import cv2
import numpy as np
from google.colab import drive

# !pip install -Uqq ipdb
# import ipdb
# %pdb on


drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
from tqdm import tqdm

**1. DATASET**

The following link has some rock scissors paper dataset you can use:

https://kaistackr-my.sharepoint.com/:f:/g/personal/shimjay17_kaist_ac_kr/EkBKx6r7RU1BnsFc5QbqFBMBHhOzF8SFoHXzaJAr1n7a3g?e=ElWBrF

To use this dataset, you will have to upload these datasets to your google drive and specify its location in the dataset path in the code below.

However, though it is not a necessity, we recommend that you test your model on additional datasets that are available to use. Remember that your model is going to be graded on an unseen custom dataset and will require your model good generalisation.

If you do decide to use an additional dataset, you may need to edit the 'loadLabels' definition accordingly in the 'Dataset' class.

In [238]:
######### Declare dataset path below! #########
#DATASET_PATH = '/content/drive/My Drive/archive/rsp'
DATASET_PATH1 = '/content/drive/MyDrive/MLintro/data_achive/archive 1/rps-cv-images'
DATASET_PATH2 = '/content/drive/MyDrive/MLintro/data_achive/archive 2'
DATASET_PATH3 = '/content/drive/MyDrive/MLintro/data_achive/mydata'
###############################################

from torchvision import transforms

class Dataset(Dataset):
    def __init__(self, dataset_dir, transform=None, target_transform=None):
        self.class_list = {'rock': 0, 'scissors': 1, 'paper': 2}
        self.labels_map = {0:'rock', 1:'scissors', 2:'paper'}
        self.path = dataset_dir #DATASET_PATH
        self.image_paths = sorted(glob.glob(os.path.join(self.path, '**', '*.jpg'), recursive=True)) + sorted(glob.glob(os.path.join(self.path, '**', '*.png'), recursive=True))
        self.transform = transforms.Compose([transforms.ToTensor()]) if transform is None else transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = self.loadImage(img_path)
        image = cv2.resize(image, (28, 28))

        # Apply the transform to the image
        if self.transform:
            image = self.transform(image)

        label = self.loadLabels(img_path)
        return image, torch.tensor(label, dtype=torch.long)  # Ensure label is a long tensor

    def loadImage(self, path):
        img = cv2.imread(path)
        img = img.astype(np.float32)/255.0  # 이미지
        return img

    def loadLabels(self, image_paths):
        class_name = os.path.dirname(image_paths).split('/')[-1]
        return self.class_list[class_name]

**2. Classification Code**

In this section, you are required to implement a network of your choice to get better performance.
You are free to edit this skeleton code however you want.

In [239]:
class CNN_BN2(nn.Module):
  def __init__(self):
    super(CNN_BN2, self).__init__()
    # Fill in the blank
    ##################################################
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=(5,5))
    self.bn1 = nn.BatchNorm2d(16)
    self.pool = nn.MaxPool2d(kernel_size=2, stride=2) # Max-pooling with kernel size = 2×2, stride = 2
    self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=5)
    self.bn2 = nn.BatchNorm2d(16)

    #####################################################

    self.fc1 = nn.Linear(in_features=256, out_features=64)
    self.bn3 = nn.BatchNorm1d(64)
    self.fc2 = nn.Linear(in_features=64, out_features=3) # The output size of last FC layer must be equal to num of class

  def forward(self, x):
    batchsize = x.size(0)
    x = self.pool(F.relu(self.bn1(self.conv1(x)))) # Convolutional layer -> BN -> ReLU activation -> max_pooling
    x = self.pool(F.relu(self.bn2(self.conv2(x))))
    x = x.view(batchsize,-1) # Change the shape of output from [batch_size, channels, height, width] to [batch_size, channels*height*width]
    x = F.relu(self.bn3(self.fc1(x)))
    out = self.fc2(x)
    return out

**3. Train & Evaluation Code**
Here is an example Train and Evaluation code for the baseline FC layer model. You will most likely have to change the code below to fit your model of choice.

In [240]:
def train(train_dataset, val_dataset, model, device='cuda:0' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.train()

    train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True) #256
    val_dataloader = DataLoader(val_dataset, batch_size=128, shuffle=False) #128
    learning_rate = 0.01 #0.001
    epoch_no = 3 #5 #10
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    print("====TRAIN INITIATED====")
    print(model)

    for epoch in tqdm(range(epoch_no)):
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for idx, data in tqdm(enumerate(train_dataloader)):
            imgs, labels = data
            imgs, labels = imgs.to(device), labels.to(device)
            output = model(imgs)

            _, predicted = torch.max(output.data, 1)
            #print("output",output)
            #print(labels)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

            model_loss = F.cross_entropy(output, labels)
            #print("output:", output)
            #print("labels:", labels)
            running_loss += model_loss.item()
            model_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        avg_train_loss = running_loss / len(train_dataloader)
        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch {epoch + 1} - Average training loss: {avg_train_loss:.3f}, Training accuracy: {train_accuracy:.2f}%")

        # validation loop every 3 epochs
        if (epoch + 1) % 3 == 0:
            val_loss = 0
            correct_val = 0
            total_val = 0
            with torch.no_grad():
                for val_idx, (val_imgs, val_labels) in enumerate(val_dataloader):
                    val_imgs, val_labels = val_imgs.to(device), val_labels.to(device)
                    val_output = model(val_imgs)

                    _, val_predicted = torch.max(val_output.data, 1)
                    total_val += val_labels.size(0)
                    correct_val += (val_predicted == val_labels).sum().item()

                    val_loss += F.cross_entropy(val_output, val_labels)

                val_accuracy = 100 * correct_val / total_val
                print(f'Validation loss: {val_loss/(val_idx+1):.3f}, Validation accuracy: {val_accuracy:.2f}%')

def test(test_dataset, model, device='cuda:0' if torch.cuda.is_available() else 'cpu'):
    model.to(device)
    model.eval()

    test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

    correct_test = 0
    total_test = 0
    test_loss = 0
    with torch.no_grad():
        for idx, (test_imgs, test_labels) in enumerate(test_dataloader):
            test_imgs, test_labels = test_imgs.to(device), test_labels.to(device)
            test_output = model(test_imgs)

            _, predicted = torch.max(test_output.data, 1)
            total_test += test_labels.size(0)
            correct_test += (predicted == test_labels).sum().item()

            test_loss += F.cross_entropy(test_output, test_labels)

        test_accuracy = 100 * correct_test / total_test
        print(f'Test loss: {test_loss/(idx+1):.3f}, Test accuracy: {test_accuracy:.2f}%')


**4. Run Code**

Run the code below to run all the whole model.

In [135]:
import random
random.seed(1)

In [241]:
if __name__ == '__main__':

    dataset = Dataset(DATASET_PATH1)
    dataset_size = len(dataset)
    print(dataset_size)
    train_size = int(dataset_size * 0.8)
    validation_size = int(dataset_size * 0.1)
    test_size = dataset_size - train_size - validation_size

    train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

    print(f"Training Data Size : {len(train_dataset)}")
    print(f"Validation Data Size : {len(validation_dataset)}")
    print(f"Testing Data Size : {len(test_dataset)}")

    model_cnn_last = CNN_BN2()
    train(train_dataset, validation_dataset, model_cnn_last)
    test(test_dataset, model_cnn_last)

2188
Training Data Size : 1750
Validation Data Size : 218
Testing Data Size : 220
====TRAIN INITIATED====
CNN_BN2(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)


  0%|          | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.23it/s]
2it [00:01,  1.19it/s]
3it [00:02,  1.21it/s]
4it [00:03,  1.19it/s]
5it [00:04,  1.23it/s]
6it [00:04,  1.20it/s]
7it [00:05,  1.24it/s]
 33%|███▎      | 1/3 [00:05<00:11,  5.66s/it]

Epoch 1 - Average training loss: 0.480, Training accuracy: 81.14%



0it [00:00, ?it/s]
1it [00:00,  1.38it/s]
2it [00:01,  1.36it/s]
3it [00:02,  1.30it/s]
4it [00:03,  1.28it/s]
5it [00:03,  1.29it/s]
6it [00:04,  1.30it/s]
7it [00:05,  1.32it/s]
 67%|██████▋   | 2/3 [00:10<00:05,  5.45s/it]

Epoch 2 - Average training loss: 0.126, Training accuracy: 96.11%



0it [00:00, ?it/s]
1it [00:00,  1.18it/s]
2it [00:01,  1.23it/s]
3it [00:02,  1.25it/s]
4it [00:03,  1.24it/s]
5it [00:04,  1.23it/s]
6it [00:04,  1.22it/s]
7it [00:05,  1.26it/s]


Epoch 3 - Average training loss: 0.061, Training accuracy: 98.17%


100%|██████████| 3/3 [00:17<00:00,  5.74s/it]

Validation loss: 0.035, Validation accuracy: 98.62%


Test loss: 0.285, Test accuracy: 91.36%


In [242]:
if __name__ == '__main__':

    dataset1 = Dataset(DATASET_PATH1)
    use_dataset1, unuse_dataset1 = random_split(dataset1, [0.54, 0.46])
    dataset2 = Dataset(DATASET_PATH2)
    print(len(dataset2))
    dataset = use_dataset1 + dataset2
    dataset_size = len(dataset)
    print(dataset_size)
    train_size = int(dataset_size * 0.8)
    validation_size = int(dataset_size * 0.1)
    test_size = dataset_size - train_size - validation_size


    train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

    print(f"Training Data Size : {len(train_dataset)}")
    print(f"Validation Data Size : {len(validation_dataset)}")
    print(f"Testing Data Size : {len(test_dataset)}")

    train(train_dataset, validation_dataset, model_cnn_last)
    test(test_dataset, model_cnn_last)


2925
4107
Training Data Size : 3285
Validation Data Size : 410
Testing Data Size : 412
====TRAIN INITIATED====
CNN_BN2(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)


  0%|          | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:00,  1.18it/s]
2it [00:01,  1.15it/s]
3it [00:02,  1.12it/s]
4it [00:03,  1.10it/s]
5it [00:04,  1.11it/s]
6it [00:05,  1.13it/s]
7it [00:06,  1.10it/s]
8it [00:07,  1.04it/s]
9it [00:08,  1.05it/s]
10it [00:09,  1.05it/s]
11it [00:10,  1.08it/s]
12it [00:11,  1.08it/s]
13it [00:11,  1.10it/s]
 33%|███▎      | 1/3 [00:11<00:23, 11.88s/it]

Epoch 1 - Average training loss: 0.750, Training accuracy: 77.29%



0it [00:00, ?it/s]
1it [00:00,  1.14it/s]
2it [00:01,  1.12it/s]
3it [00:02,  1.10it/s]
4it [00:03,  1.12it/s]
5it [00:04,  1.13it/s]
6it [00:05,  1.12it/s]
7it [00:06,  1.13it/s]
8it [00:07,  1.13it/s]
9it [00:08,  1.10it/s]
10it [00:09,  1.05it/s]
11it [00:10,  1.05it/s]
12it [00:11,  1.05it/s]
13it [00:11,  1.11it/s]
 67%|██████▋   | 2/3 [00:23<00:11, 11.80s/it]

Epoch 2 - Average training loss: 0.202, Training accuracy: 95.25%



0it [00:00, ?it/s]
1it [00:00,  1.08it/s]
2it [00:01,  1.11it/s]
3it [00:02,  1.11it/s]
4it [00:03,  1.14it/s]
5it [00:04,  1.13it/s]
6it [00:05,  1.14it/s]
7it [00:06,  1.09it/s]
8it [00:07,  1.07it/s]
9it [00:08,  1.08it/s]
10it [00:09,  1.07it/s]
11it [00:10,  1.06it/s]
12it [00:11,  1.05it/s]
13it [00:11,  1.09it/s]


Epoch 3 - Average training loss: 0.102, Training accuracy: 97.50%


100%|██████████| 3/3 [00:37<00:00, 12.39s/it]

Validation loss: 0.129, Validation accuracy: 98.78%


Test loss: 0.084, Test accuracy: 97.82%


In [243]:
# 모델 전체 저장
PATH = '/content/drive/MyDrive/MLintro/last.pth'
torch.save(model_cnn_last, PATH)

In [245]:
model_cnn_last =  torch.load('/content/drive/MyDrive/MLintro/last.pth')

In [247]:
if __name__ == '__main__':
    dataset1 = Dataset(DATASET_PATH1)
    use_dataset1, unuse_dataset1 = random_split(dataset1, [0.32, 1-0.32])
    dataset2 = Dataset(DATASET_PATH2)
    use_dataset2, unuse_dataset2 = random_split(dataset2, [0.24, 1-0.24])

    dataset3 = Dataset(DATASET_PATH3)
    dataset = use_dataset1 + use_dataset2 + dataset3
    print(len(dataset3))
    dataset_size = len(dataset)
    print(dataset_size)
    train_size = int(dataset_size * 0.8)
    validation_size = int(dataset_size * 0.1)
    test_size = dataset_size - train_size - validation_size


    train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size])

    print(f"Training Data Size : {len(train_dataset)}")
    print(f"Validation Data Size : {len(validation_dataset)}")
    print(f"Testing Data Size : {len(test_dataset)}")

    train(train_dataset, validation_dataset, model_cnn_last)
    test(test_dataset, model_cnn_last)

2746
4149
Training Data Size : 3319
Validation Data Size : 414
Testing Data Size : 416
====TRAIN INITIATED====
CNN_BN2(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(16, 16, kernel_size=(5, 5), stride=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
)


  0%|          | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [04:02, 242.27s/it]
2it [07:37, 226.08s/it]
3it [11:09, 219.75s/it]
4it [14:13, 205.63s/it]
5it [17:30, 202.62s/it]
6it [20:53, 202.74s/it]
7it [24:09, 200.67s/it]
8it [27:22, 198.24s/it]
9it [31:03, 205.34s/it]
10it [34:36, 207.64s/it]
11it [38:14, 210.73s/it]
12it [42:08, 217.70s/it]
13it [45:40, 210.80s/it]
 33%|███▎      | 1/3 [45:40<1:31:20, 2740.38s/it]

Epoch 1 - Average training loss: 0.717, Training accuracy: 76.59%



0it [00:00, ?it/s]
1it [00:35, 35.21s/it]
2it [01:08, 34.20s/it]
3it [01:39, 32.81s/it]
4it [02:12, 32.90s/it]
5it [02:47, 33.69s/it]
6it [03:21, 33.63s/it]
7it [03:55, 33.88s/it]
8it [04:31, 34.32s/it]
9it [05:04, 33.98s/it]
10it [05:38, 34.07s/it]
11it [06:14, 34.52s/it]
12it [06:51, 35.31s/it]
13it [07:27, 34.41s/it]
 67%|██████▋   | 2/3 [53:07<23:11, 1391.52s/it]  

Epoch 2 - Average training loss: 0.271, Training accuracy: 91.93%



0it [00:00, ?it/s]
1it [00:30, 30.82s/it]
2it [01:06, 33.77s/it]
3it [01:41, 34.30s/it]
4it [02:15, 34.31s/it]
5it [02:51, 34.72s/it]
6it [03:24, 34.34s/it]
7it [04:01, 35.14s/it]
8it [04:36, 35.16s/it]
9it [05:12, 35.22s/it]
10it [05:46, 34.97s/it]
11it [06:22, 35.33s/it]
12it [06:53, 33.79s/it]
13it [07:26, 34.37s/it]


Epoch 3 - Average training loss: 0.103, Training accuracy: 97.68%


100%|██████████| 3/3 [1:07:19<00:00, 1346.35s/it]

Validation loss: 0.081, Validation accuracy: 98.31%


Test loss: 0.183, Test accuracy: 94.23%


In [260]:
# 모델 전체 저장
PATH = '/content/drive/MyDrive/MLintro/last1.pth'
torch.save(model_cnn_last, PATH)

In [280]:
# 모델 전체 로드
loaded_model = torch.load('/content/drive/MyDrive/MLintro/last1.pth')

**5. Inference**

Once you are satisfied with your classifier code, you are to edit the code below so that it runs. The code below should successfully be able to load the 'test_image.jpeg' within the 'Project items' folder in the link given above and give a prediction for the image using your proposed model.

The final output should return either 'rock', 'scissors' or 'paper'.

In [ ]:
loaded_model = torch.load(PATH)

In [286]:
#chosen_img_path = '/content/drive/My Drive/archive/inference\test_image_jpeg'
chosen_img_path = '/content/drive/MyDrive/MLintro/hand_test/test_image.jpeg'
################ Edit Here ################
from torchvision import transforms


img = cv2.imread(chosen_img_path)
img = img.astype(np.float32)/255.0
img = cv2.resize(img, (28, 28))

transform = transforms.Compose([transforms.ToTensor()])
input = transform(img)
input = input.to(device='cuda:0' if torch.cuda.is_available() else 'cpu')

input = input.unsqueeze(0)
loaded_model.eval()
################ Edit Here ################

output = loaded_model(input)

_, predicted_class = torch.max(output, 1)

class_labels = ['rock', 'scissors', 'paper']
predicted_label = class_labels[predicted_class.item()]
print(predicted_label)


scissors


In [288]:
PATH = '/content/drive/MyDrive/MLintro/Kimgeonwoo_20200045.pth'
torch.save(loaded_model.state_dict(), PATH)